In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from fastparquet import ParquetFile
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
DATA_FOLDER = 'data/competition_data_final_pqt'

### Препроцессинг

Делаем словарь устройство - цена, чтобы заполнить недостающие значения:

In [ ]:
prices_data = []
for file in tqdm(os.listdir(DATA_FOLDER)):
    parquet = ParquetFile(os.path.join(DATA_FOLDER, file))
    data = parquet.to_pandas(['cpe_model_name', 'price'])
    prices_data.append(data)
models2prices = pd.concat(prices_data, axis=0)

models2prices = models2prices.groupby('cpe_model_name')['price'].first().round().to_dict()

In [ ]:
def preprocess_data(data):

    data['date'] = pd.to_datetime(data['date'])
    data['day'] = data['date'].dt.day
    data['month'] = data['date'].dt.month
    data['year'] = data['date'].dt.year

    data = data.sort_values(by=['user_id', 'date'])

    data['price'] = data['cpe_model_name'].map(models2prices)

    return data

In [ ]:
for i, file in tqdm(enumerate(os.listdir(DATA_FOLDER))):
    parquet = ParquetFile(os.path.join(DATA_FOLDER, file))
    data = parquet.to_pandas()
    preprocessed_data = preprocess_data(data)
    preprocessed_data.to_parquet(f'preprocessed2/data_part{i}.parquet', engine='fastparquet')

### Аггрегация

Аггрегируем новые признаки:
- статистики по кол-ву запросов
- кол-во уникальных значений посещенных сайтов, городов и регионов
- мода по категориальным переменным
- среднее и максимум по цене устройства
- длительность периода активности пользователя
- отношение количества запросов к длительности
- кол-во запросов в среднем за день / месяц
- доли запросов за утро / день / вечер / ночь
- кол-во дней без запросов

In [ ]:
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

agg_func = {'price': ['mean', 'max'], 
                'request_cnt': ['mean', 'median', 'std', 'min', 'max', 'sum'],
                'region_name': ['nunique', get_mode],
                'city_name': ['nunique', get_mode], 
                'cpe_manufacturer_name': [get_mode], 
                'cpe_type_cd': [get_mode], 
                'cpe_model_os_type': [get_mode], 
                'date': ['first', 'last'],
                'url_host': ['nunique']}

In [ ]:
def aggregate_features(data):

    table = pd.pivot_table(data, values=['price', 'request_cnt', 'region_name', 'city_name', 'date',
                                        'cpe_manufacturer_name', 'cpe_type_cd', 'cpe_model_os_type', 
                                        'cpe_model_name', 'url_host'], 
                            index=['user_id'], aggfunc=agg_func)

    table.columns = table.columns.map(' '.join)
    table['duration'] = (table['date last'] - table['date first']) / np.timedelta64(1, 'D')
    table = table.drop(['date first', 'date last'], axis=1)
    table['frequency'] = table['duration'] / table['request_cnt sum']

    month_requests = data.groupby(['user_id', 'year', 'month'])['request_cnt'].agg('sum').to_frame(name='avg_month')
    avg_month_requests = month_requests.groupby('user_id').agg('mean')
    day_requests = data.groupby(['user_id', 'year', 'month', 'day'])['request_cnt'].agg('sum').to_frame(name='avg_day')
    avg_day_requests = day_requests.groupby('user_id').agg('mean')

    data['days_since_prev'] = data.groupby('user_id')['date'].diff().dt.days
    churn_1 = data[data['days_since_prev'] > 1].groupby('user_id')['days_since_prev'].count().to_frame(name='churn1')
    churn_10 = data[data['days_since_prev'] > 10].groupby('user_id')['days_since_prev'].count().to_frame(name='churn10')
    churn_30 = data[data['days_since_prev'] > 30].groupby('user_id')['days_since_prev'].count().to_frame(name='churn30')

    df_pod = pd.pivot_table(data, values='request_cnt', index=['user_id'], columns=['part_of_day'], aggfunc=np.sum).reset_index().fillna(0)

    total_pod = df_pod['day'] + df_pod['evening'] + df_pod['morning'] + df_pod['night'] + 1
    df_pod['day'] /= total_pod
    df_pod['evening'] /= total_pod
    df_pod['morning'] /= total_pod
    df_pod['night'] /= total_pod

    return pd.concat([table, avg_month_requests, df_pod, avg_day_requests,
                     churn_1, churn_10, churn_30], axis=1)

In [ ]:
preproc_folder = 'preprocessed2'

all_agg_data = []
for i, file in enumerate(os.listdir(preproc_folder)):
    parquet = ParquetFile(os.path.join(preproc_folder, file))
    data = parquet.to_pandas(['user_id', 'request_cnt', 'date'])
    all_agg_data.append(aggregate_features(data))
train_features = pd.concat(all_agg_data)

In [ ]:
train_features = train_features.fillna(0)

In [ ]:
cat_columns = ['city_name get_mode', 'cpe_manufacturer_name get_mode', 'cpe_model_os_type get_mode',
                'cpe_type_cd get_mode',  'region_name get_mode']

label_enc = LabelEncoder()

for feature in cat_columns:
    train_features[feature] = label_enc.fit_transform(train_features[feature])

In [ ]:
train_features.to_csv('data/train_features.csv', index=False)